In [3]:
pip install cuda-python torch torchvision torchaudio Cython --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
ERROR: Could not find a version that satisfies the requirement colab_transfer (from versions: none)
ERROR: No matching distribution found for colab_transfer


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir "/content/drive2/"
%cp -av "/content/drive/MyDrive/data" "/content/drive2/"
%cp -av "/content/drive/MyDrive/garbage_files/" "/content/drive2/"

Mounted at /content/drive
'/content/drive/MyDrive/data' -> '/content/drive2/data'
'/content/drive/MyDrive/data/glass' -> '/content/drive2/data/glass'
'/content/drive/MyDrive/data/glass/glass19.jpg' -> '/content/drive2/data/glass/glass19.jpg'
'/content/drive/MyDrive/data/glass/glass397.jpg' -> '/content/drive2/data/glass/glass397.jpg'
'/content/drive/MyDrive/data/glass/glass123.jpg' -> '/content/drive2/data/glass/glass123.jpg'
'/content/drive/MyDrive/data/glass/glass232.jpg' -> '/content/drive2/data/glass/glass232.jpg'
'/content/drive/MyDrive/data/glass/glass32.jpg' -> '/content/drive2/data/glass/glass32.jpg'
'/content/drive/MyDrive/data/glass/glass414.jpg' -> '/content/drive2/data/glass/glass414.jpg'
'/content/drive/MyDrive/data/glass/glass18.jpg' -> '/content/drive2/data/glass/glass18.jpg'
'/content/drive/MyDrive/data/glass/glass342.jpg' -> '/content/drive2/data/glass/glass342.jpg'
'/content/drive/MyDrive/data/glass/glass307.jpg' -> '/content/drive2/data/glass/glass307.jpg'
'/content/

In [3]:
import os
import torch
import torchvision
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import make_grid
import numpy as np

def load_data(path: str):
    with open(path, "r") as f:
        files = f.readlines()
        targets = []
        images = []

        for line in files:
            name, target = line.split(" ")
            if name.startswith("paper"):
                path = f"/content/drive2/data/paper/{name}"
            elif name.startswith("glass"):
                path = f"/content/drive2/data/glass/{name}"
            elif name.startswith("metal"):
                path = f"/content/drive2/data/metal/{name}"
            elif name.startswith("cardboard"):
                path = f"/content/drive2/data/cardboard/{name}"
            elif name.startswith("plastic"):
                path = f"/content/drive2/data/plastic/{name}"
            elif name.startswith("trash"):
                path = f"/content/drive2/data/trash/{name}"

            targets.append(int(target)-1)
            image = torchvision.io.read_image(path)
            images.append(image)

    return images, targets



class TrashDataset(Dataset):

  def __init__(self, images, targets):
    self.targets = targets
    self.images = images

  def __len__(self):
    return len(self.targets)

  transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor()])

  def __getitem__(self, index):
    return self.targets[index], self.transform(self.images[index])


class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = torchvision.models.vgg19(pretrained=True)
    self.fc1 = nn.Linear(1000, 6)
  def forward(self, x):
    x = self.network(x)
    x = self.fc1(x)
    return x

In [ ]:
import os
import torch
import torchvision
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import make_grid
import numpy as np


def load_data(path: str):
    with open(path, "r") as f:
        files = f.readlines()
        targets = []
        images = []

        for line in files:
            name, target = line.split(" ")
            if name.startswith("paper"):
                path = f"/content/drive2/data/paper/{name}"
            elif name.startswith("glass"):
                path = f"/content/drive2/data/glass/{name}"
            elif name.startswith("metal"):
                path = f"/content/drive2/data/metal/{name}"
            elif name.startswith("cardboard"):
                path = f"/content/drive2/data/cardboard/{name}"
            elif name.startswith("plastic"):
                path = f"/content/drive2/data/plastic/{name}"
            elif name.startswith("trash"):
                path = f"/content/drive2/data/trash/{name}"

            targets.append(int(target)-1)
            image = torchvision.io.read_image(path)
            images.append(image)

    return images, targets


class TrashDataset(Dataset):

  def __init__(self, images, targets):
    self.targets = targets
    self.images = images

  def __len__(self):
    return len(self.targets)

  transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor()])

  def __getitem__(self, index):
    return self.targets[index], self.transform(self.images[index])


class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = torchvision.models.vgg16(pretrained=True)
    self.fc1 = nn.Linear(1000, 6)
  def forward(self, x):
    x = self.network(x)
    x = self.fc1(x)
    return x


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 40
cv_scores = []

train_images, train_targets = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_train.txt")
train_dataset = TrashDataset(images=train_images, targets=train_targets)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

cv_images, cv_targets = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_val.txt")
cv_dataset = TrashDataset(images=cv_images, targets=cv_targets)
cv_loader = torch.utils.data.DataLoader(cv_dataset, batch_size=batch_size, shuffle=True, num_workers=0)


net = Net()
net.to(device)

#num_ftrs = 512*7*7
#model_ft.fc = nn.Linear(num_ftrs, 2)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

for epoch in range(20):
    for idx, data in enumerate(train_loader):
        targets, inputs = data
        targets, inputs = targets.cuda(), inputs.cuda()

        optimizer_ft.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer_ft.step()
        print(f'[{epoch + 1}, {idx + 1:5d}] loss: {loss}')

        correct = 0
        total = 0
    with torch.no_grad():
        for data in cv_loader:
            targets, inputs = data
            targets, inputs = targets.cuda(), inputs.cuda()
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        print(f'Accuracy of the network on the cv images: {100 * correct // total} %')
        
        torch.save(net, "/content/drive/MyDrive/models/epoch.max")

        cv_scores.append(100 * correct // total)



    
print('Finished Training')
print(cv_scores)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

[1,     1] loss: 2.784825086593628
[1,     2] loss: 1.9232546091079712
[1,     3] loss: 1.8352835178375244
[1,     4] loss: 1.476261854171753
[1,     5] loss: 1.7155191898345947
[1,     6] loss: 1.6009714603424072
[1,     7] loss: 1.4625837802886963
[1,     8] loss: 1.2460181713104248
[1,     9] loss: 1.3878147602081299
[1,    10] loss: 1.4479281902313232
[1,    11] loss: 1.1286309957504272
[1,    12] loss: 1.2655876874923706
[1,    13] loss: 1.09933602809906
[1,    14] loss: 0.903835654258728
[1,    15] loss: 1.0999490022659302
[1,    16] loss: 1.157639503479004
[1,    17] loss: 1.037042498588562
[1,    18] loss: 1.1963177919387817
[1,    19] loss: 0.7827540636062622
[1,    20] loss: 0.8028141856193542
[1,    21] loss: 0.8633324503898621
[1,    22] loss: 0.9005594253540039
[1,    23] loss: 0.9632871747016907
[1,    24] loss: 1.0579472780227661
[1,    25] loss: 0.9129984974861145
[1,    26] loss: 0.9817021489143372
[1,    27] loss: 1.0607995986938477
[1,    28] loss: 0.8825944662094116

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = ['Glas', 'Papier', 'Pappe', 'Plastik', 'Metall', 'Müll']
batch_size = 64
net = Net()
net.to(device)

model = torch.load("/content/drive/MyDrive/models/epoch.max")
try_image, try_target = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_test.txt")
try_dataset = TrashDataset(images=try_image, targets=try_target)
try_loader = torch.utils.data.DataLoader(try_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

"""
for data in try_loader:
  targets, inputs = data
  #targets, inputs = targets.cuda(), inputs.cuda()
  outputs = net(inputs)
  _, predicted = torch.max(outputs.data,1)
  print(f"Der Müll auf dem Bild ist {classes[predicted]}")
"""
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for data in try_loader:
    targets, inputs = data
    targets, inputs = targets.cuda(), inputs.cuda()
    # calculate outputs by running images through the network
    outputs = model(inputs)
    # the class with the highest energy is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()
  print(f'Accuracy of the network on the test images: {100 * correct // total} %')